In [1]:
import getpass
import os
from neo4j import GraphDatabase

from dotenv import load_dotenv
import os

# Load variables from .env into environment
load_dotenv()

# Access them
NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
print("Connecting to Neo4j at:", NEO4J_URI)

def check_driver_ready():
    try:
        with driver.session() as session:
            result = session.run("RETURN 1")
            value = result.single()[0]
            if value == 1:
                print("✅ Neo4j driver is ready!")
                return True
    except Exception as e:
        print(f"❌ Neo4j driver is not ready: {e}")
        return False

# Call it
check_driver_ready()

Connecting to Neo4j at: neo4j://127.0.0.1:7687
✅ Neo4j driver is ready!


True

In [3]:
import csv
def create_entity(tx, entity_name, entity_type):
    tx.run("""
           MERGE (e:Entity {name: $name, type: $type})
           """, name=entity_name, type=entity_type
           )

def create_relationship(tx, src_name, tgt_name, rel_type):
    tx.run("""
           MATCH (a:Entity {name: $src_name})
           MATCH (b:Entity {name: $tgt_name})
           MERGE (a)-[r:RELATION {type: $rel_type}] -> (b)
            """, src_name = src_name, tgt_name = tgt_name, rel_type=rel_type)
    
with driver.session() as session:
    with open('triple.csv', newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            session.write_transaction(create_entity, row['src_entity'], row['src_type'])
            session.write_transaction(create_entity, row['tgt_entity'], row['tgt_type'])
            session.write_transaction(create_relationship, row['src_entity'], row['tgt_entity'], row['rel_type'])



/var/folders/1d/vmp1xyxn71j6xhr6qw60f1m00000gn/T/ipykernel_80014/1177847170.py:19: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_entity, row['src_entity'], row['src_type'])
/var/folders/1d/vmp1xyxn71j6xhr6qw60f1m00000gn/T/ipykernel_80014/1177847170.py:20: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_entity, row['tgt_entity'], row['tgt_type'])
/var/folders/1d/vmp1xyxn71j6xhr6qw60f1m00000gn/T/ipykernel_80014/1177847170.py:21: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_relationship, row['src_entity'], row['tgt_entity'], row['rel_type'])


In [2]:
from langchain_neo4j import Neo4jGraph

graph = Neo4jGraph()

# Import movie information

movies_query = """
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv'
AS row
MERGE (m:Movie {id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))
"""

graph.query(movies_query)

ClientError: {code: Neo.ClientError.Procedure.ProcedureRegistrationFailed} {message: apoc.meta.data is unavailable because it is sandboxed and has dependencies outside of the sandbox. Sandboxing is controlled by the dbms.security.procedures.unrestricted setting. Only unrestrict procedures you can trust with access to database internals.}